#Output Parser DataFrame

In [ ]:
!pip install -U langchain-openai langchain

In [1]:
#define constante 
OPENAI_API_KEY= "YOUR_API_KEY"

In [4]:
import pprint
from typing import Any, Dict

import pandas as pd
from langchain.output_parsers import PandasDataFrameOutputParser
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

model = ChatOpenAI(temperature=0,api_key=OPENAI_API_KEY)

# Solely for documentation purposes.
def format_parser_output(parser_output: Dict[str, Any]) -> None:
    """
    Retourne la valeur de la requête sans print le dictionnaire
    """
    for key in parser_output.keys():
        parser_output[key] = parser_output[key].to_dict()
    return pprint.PrettyPrinter(width=4, compact=True).pprint(parser_output)

# Define your desired Pandas DataFrame.
df = pd.DataFrame(
    {
        "num_legs": [2, 4, 8, 0],
        "num_wings": [2, 0, 0, 0],
        "num_specimen_seen": [10, 2, 1, 8],
    }
)

# Set up a parser + inject instructions into the prompt template.
parser = PandasDataFrameOutputParser(dataframe=df)


In [3]:
df

,num_legs,num_wings,num_specimen_seen
0,2,2,10
1,4,0,2
2,8,0,1
3,0,0,8


In [5]:
# Here's an example of a column operation being performed.
df_query = "Donne-moi la colonne 'num_wings'."

# Set up the prompt.
prompt = PromptTemplate(
    template="Réponds à la requète de l'utilisateur.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | model | parser
parser_output = chain.invoke({"query": df_query})

format_parser_output(parser_output)

{'num_wings': {0: 2,
               1: 0,
               2: 0,
               3: 0}}


In [6]:
# Here's an example of a row operation being performed.
df_query = "Retourne-moi la première ligne."

chain = prompt | model | parser
parser_output = chain.invoke({"query": df_query})

format_parser_output(parser_output)

{'0': {'num_legs': 2,
       'num_specimen_seen': 10,
       'num_wings': 2}}


In [7]:
# Here's an example of a row operation being performed.
df_query = "Retourne-moi la moyenne de la colonne 'num_legs' de la ligne 1 à 3."

chain = prompt | model | parser
parser_output = chain.invoke({"query": df_query})

parser_output

{'mean': 4.0}